# NLP With Disaster Tweets

## Imports

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 1000)

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import nltk

# import sklearn.model_selection import train_test_split
# from sklearn.metrics import f1_score

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [ ]:
train_df['text'].sample(5)

In [ ]:
train_df['text'][train_df.index == 7146]

In [ ]:
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub (r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

In [ ]:
train_df['cleaned_text'] = train_df['text'].apply(preprocess_text)
test_df['cleaned_text'] = test_df['text'].apply(preprocess_text)

In [ ]:
train_df.head()

In [ ]:
train_df['cleaned_text'][train_df.index == 7146]

## Data Visualization

In [ ]:
plt.style.use('fivethirtyeight')
sns.countplot(x = 'target', data = train_df, palette = ['salmon', 'purple'])
plt.title('Distribution of target')
plt.show()

In [ ]:
train_df['text_length'] = train_df['text'].apply(len)
sns.histplot(train_df['text_length'], bins = 40, color = 'darkblue')
plt.title('Distribution of tweet lengths')
plt.show()

## Data Preprocessing

## Model Building